# Random hexamer barcode collapsing

In this notebook I collapse random hexamers from starsolo output. Then, I perform QC and the whole data (poliT and random hexamers) and only in poliT primer subset of the data. I then cluster all the cells with all the data and only with poliT information. This notebook is coded for one species at a time (for species integration go to "testis_singlecell/Workspaces/mtxellrb/to_paula/cluster_integration.ipynb")

In [1]:
import anndata as ad
import scanpy as sc
import seaborn as sns
import pandas as pd
import numpy as np
import bbknn
import matplotlib.pyplot as plt
import skmisc
import os
# import scvi

from IPython.display import Markdown, display
from IPython.display import display, HTML
from os import environ

sns.set_context('poster')

display(HTML("<style>.container { width:90% !important; }</style>"))
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80) #resolution

In [2]:
def load_barcode_replacements(barcode_info):
    """Load the barcode(1) (R) -> barcode(1) R replacement list"""
    # barcode_info = pd.read_csv(barcode_path / f"bc_data_{barcode_name}.csv", sep=",")
    # We combine T and R into T
    # to be identical to splitpipe's approach.
    pairs_by_well = {}
    for _, row in barcode_info.iterrows():
        well = row["well"]
        if not well in pairs_by_well:
            pairs_by_well[well] = [None, None]
        if row["stype"] == "T":
            pairs_by_well[well][0] = row["sequence"]
        else:
            pairs_by_well[well][1] = row["sequence"]
    replacements = {x[1]: x[0] for x in pairs_by_well.values()}
    # assert len(replacements) == len(barcode_info) / 2
    return replacements

In [3]:
def get_adata(path_to_mtx,samples):
    sample = samples[0]

    adata = sc.read_10x_mtx(path_to_mtx[0],cache=True)
    adata.obs_names = [ sample+"_"+x.split("-")[0] for x in adata.obs_names ]
    adata.obs["DATASET"] = sample

    for index,file in enumerate(path_to_mtx[1:]):
        sample = samples[index+1]

        adata_new = sc.read_10x_mtx(file,cache=True)
        adata_new.obs_names = [ sample+"_"+x.split("-")[0] for x in adata_new.obs_names ]
        adata_new.obs["DATASET"] = sample

        adata = adata.concatenate(adata_new,index_unique=None)
        del(adata_new)
        
    return adata

First round of barcodes (linked to poliT or to random hexamers) used for v1 WT kit

In [4]:
bc_1 = "/home/paulilokiestudia/testis_singlecell/Workspaces/paula/data/barcode_whitelists/bc_data_v2.csv"

In [5]:
df = pd.read_csv(bc_1, delimiter=',')

In [6]:
sp = "WCG"
samples = "Whitecheeked_gibbon".split(",")
zu = 4

In [7]:
path_to_mtx_starsolo_list = []
path_to_mapping = f'/home/paulilokiestudia/testis_singlecell/Workspaces/paula/starsolo_v1/{sp}/'########
for sample in samples:
    path_to_mtx_starsolo_list.append(f'{path_to_mapping}{sample}/combined_fil/')

In [8]:
path_out = f'/home/paulilokiestudia/testis_singlecell/Workspaces/paula/starsolo_v1/{sp}/adata'

if not os.path.exists(path_out):
    os.makedirs(path_out)

adata_out_mm = f'{path_out}'

adata_out_mm = f'{path_out}/preproc_1_starsolo_multimap_collapsed.h5ad'

In [9]:
adata = get_adata(path_to_mtx_starsolo_list,samples)

... writing an h5ad cache file to speedup reading next time


In [10]:
adata

AnnData object with n_obs × n_vars = 7576 × 34643
    obs: 'DATASET'
    var: 'gene_ids', 'feature_types'

In [11]:
cell_names = adata.obs.index
print(cell_names[-5:])

Index(['Whitecheeked_gibbon_AGATCGCA_TGAAGAGA_TTCATCGC_PARSE6_UDI_WT_4',
       'Whitecheeked_gibbon_CAATGGAA_TGGTGGTA_TTCATCGC_PARSE6_UDI_WT_4',
       'Whitecheeked_gibbon_CCTCCTGA_TGGTGGTA_TTCATCGC_PARSE6_UDI_WT_4',
       'Whitecheeked_gibbon_ACAGCAGA_TTCACGCA_TTCATCGC_PARSE6_UDI_WT_4',
       'Whitecheeked_gibbon_TATCAGCA_TTCACGCA_TTCATCGC_PARSE6_UDI_WT_4'],
      dtype='object')


In [12]:
bc_matched = load_barcode_replacements(df)

In [13]:
for key in list(bc_matched)[:5]:
    print(key, bc_matched[key])

CTGCTTTG ACTCGTAA
CATGATCA AAACGATA
GGGTAGCG TTACCTCG
CCGAGAAA GCCTGCAA
ACGGACTC TGGTATAC


In [14]:
# bc_matched

In [15]:
poliT_primers = set(bc_matched.values())
randomhexprimers = set(bc_matched.keys())

In [16]:
# poliT_primers_list = list(poliT_primers)

# with open('/home/paulilokiestudia/testis_singlecell/Workspaces/paula/data/barcode_whitelists/bc_data_v2_poliT.txt', 'w') as file:
#     for primer in poliT_primers_list:
#         file.write(f"{primer}\n")

In [17]:
# poliT_primers_list = list(randomhexprimers)

# with open('/home/paulilokiestudia/testis_singlecell/Workspaces/paula/data/barcode_whitelists/bc_data_v2_randomhexamer.txt', 'w') as file:
#     for primer in poliT_primers_list:
#         file.write(f"{primer}\n")

In [18]:
bc_values = set(bc_matched.keys())

matching_cells = []

for cell_name in adata.obs.index:
    parts = cell_name.split('_')
    # probably to generalize it to other species i can remove the sample name, then split it, and take parts[0]
    # because Chimpanzee_stephan sample has only 1 '_', but Brown_Wooly_Monkey_1 has 3 '_'
    # Check if the second section is in bc_matched values
    if parts[zu] in bc_values:
        matching_cells.append(cell_name)

In [19]:
# Output matching cells
print(matching_cells[:2])

['Whitecheeked_gibbon_CCAGTTCA_AAACATCG_AGGATTAA_PARSE1_UDI_WT_1', 'Whitecheeked_gibbon_AACCGAGA_AACGCTTA_AGGATTAA_PARSE1_UDI_WT_1']


In [20]:
len(matching_cells)

1322

#### Found barcodes match perfectly with provided barcodes

In [21]:
primer_types = []

# Iterate through the cell names in adata.obs.index
for cell_name in adata.obs.index:
    # Split the cell name by underscore
    parts = cell_name.split('_')
    
    # Check if parts[6] is in the keys or values of bc_matched
    if parts[zu] in bc_matched.values():
        primer_types.append('T')  # If in keys, set primer_type to 'T'
    elif parts[zu] in bc_matched.keys():
        primer_types.append('R')  # If in values, set primer_type to 'R'
    else:
        primer_types.append(None)  # No match, you can assign None or another value

# Add the new column to the obs DataFrame
adata.obs['primer_type'] = primer_types

In [22]:
print(adata.obs[['primer_type']])

                                                   primer_type
Whitecheeked_gibbon_CCAGTTCA_AAACATCG_AGGATTAA_...           R
Whitecheeked_gibbon_AACCGAGA_AACGCTTA_AGGATTAA_...           R
Whitecheeked_gibbon_CAACCACA_AAGACGGA_AGGATTAA_...           R
Whitecheeked_gibbon_GGTGCGAA_AAGACGGA_AGGATTAA_...           R
Whitecheeked_gibbon_GTCTGTCA_AAGACGGA_AGGATTAA_...           R
...                                                        ...
Whitecheeked_gibbon_AGATCGCA_TGAAGAGA_TTCATCGC_...           R
Whitecheeked_gibbon_CAATGGAA_TGGTGGTA_TTCATCGC_...           R
Whitecheeked_gibbon_CCTCCTGA_TGGTGGTA_TTCATCGC_...           R
Whitecheeked_gibbon_ACAGCAGA_TTCACGCA_TTCATCGC_...           R
Whitecheeked_gibbon_TATCAGCA_TTCACGCA_TTCATCGC_...           R

[7576 rows x 1 columns]


In [23]:
adata.obs['primer_type'].isna().sum()

0

In [24]:
# Create a summary table counting 'T' and 'R'
summary_table = adata.obs['primer_type'].value_counts().reset_index()
summary_table.columns = ['primer_type', 'count']

print(summary_table)

  primer_type  count
0           T   6254
1           R   1322


In [25]:
adata.obs['primer_type'].isna().sum()

0

#### Save file with only poliT primers

In [26]:
filtered_adata = adata[adata.obs['primer_type'] == 'T'].copy()

In [27]:
# filtered_adata.write(f'{path_out}/preproc_1_starsolo_multimap_only_poliT.h5ad')

### Save all data 

Collapse random hexamers

In [28]:
adata_R = adata[adata.obs['primer_type'] == 'R'].copy()

In [29]:
adata_R

AnnData object with n_obs × n_vars = 1322 × 34643
    obs: 'DATASET', 'primer_type'
    var: 'gene_ids', 'feature_types'

In [30]:
adata_R.obs[-5:]

,DATASET,primer_type
Whitecheeked_gibbon_AGATCGCA_TGAAGAGA_TTCATCGC_PARSE6_UDI_WT_4,Whitecheeked_gibbon,R
Whitecheeked_gibbon_CAATGGAA_TGGTGGTA_TTCATCGC_PARSE6_UDI_WT_4,Whitecheeked_gibbon,R
Whitecheeked_gibbon_CCTCCTGA_TGGTGGTA_TTCATCGC_PARSE6_UDI_WT_4,Whitecheeked_gibbon,R
Whitecheeked_gibbon_ACAGCAGA_TTCACGCA_TTCATCGC_PARSE6_UDI_WT_4,Whitecheeked_gibbon,R
Whitecheeked_gibbon_TATCAGCA_TTCACGCA_TTCATCGC_PARSE6_UDI_WT_4,Whitecheeked_gibbon,R


In [31]:
new_obs_names = []
for cell_name in adata_R.obs.index:
    parts = cell_name.split('_')
    if parts[4] in bc_matched.keys():
        #replace parts[2] with corresponding key
        for key, value in bc_matched.items():
            if key == parts[4]:
                parts[4] = value
                break
        #reconstruct new cellname
        new_cell_name = '_'.join(parts)
        new_obs_names.append(new_cell_name)
    else:
        new_obs_names.append(cell_name) 

In [32]:
len(new_obs_names)

1322

In [33]:
# update rownames of adata_matching with updated names
adata_R.obs_names = new_obs_names

In [34]:
adata_R.obs_names[:5]

Index(['Whitecheeked_gibbon_CCAGTTCA_AAACATCG_GTGCTAGC_PARSE1_UDI_WT_1',
       'Whitecheeked_gibbon_AACCGAGA_AACGCTTA_GTGCTAGC_PARSE1_UDI_WT_1',
       'Whitecheeked_gibbon_CAACCACA_AAGACGGA_GTGCTAGC_PARSE1_UDI_WT_1',
       'Whitecheeked_gibbon_GGTGCGAA_AAGACGGA_GTGCTAGC_PARSE1_UDI_WT_1',
       'Whitecheeked_gibbon_GTCTGTCA_AAGACGGA_GTGCTAGC_PARSE1_UDI_WT_1'],
      dtype='object')

In [35]:
adata_matching_sorted = adata_R[adata_R.obs_names.argsort()].copy()
adata_remaining_sorted = filtered_adata[filtered_adata.obs_names.argsort()].copy()

In [36]:
obs_names_adata1 = set(adata_matching_sorted.obs_names)
obs_names_adata2 = set(adata_remaining_sorted.obs_names)

# Find the common obs_names between the two datasets
common_obs_names = obs_names_adata1.intersection(obs_names_adata2)

# Print the common obs_names and the count
print(f"Number of common obs_names: {len(common_obs_names)}")

Number of common obs_names: 1320


In [37]:
adata_matching_sorted

AnnData object with n_obs × n_vars = 1322 × 34643
    obs: 'DATASET', 'primer_type'
    var: 'gene_ids', 'feature_types'

In [38]:
adata_matching_sorted = adata_matching_sorted[adata_matching_sorted.obs_names.isin(common_obs_names)].copy()

In [39]:
adata_matching_sorted

AnnData object with n_obs × n_vars = 1320 × 34643
    obs: 'DATASET', 'primer_type'
    var: 'gene_ids', 'feature_types'

In [40]:
adata_remaining_sorted

AnnData object with n_obs × n_vars = 6254 × 34643
    obs: 'DATASET', 'primer_type'
    var: 'gene_ids', 'feature_types'

Take out the cells from adata_remaining_sorted that are the same as in the adata_matching sorted.
Then, sum the matrices (because the rownames and colnames are the same)
Then, concatenate it with 

In [41]:
adata_solo_cells = filtered_adata[~filtered_adata.obs_names.isin(common_obs_names)].copy()
adata_remaining_subset = filtered_adata[filtered_adata.obs_names.isin(common_obs_names)].copy()

In [42]:
adata_solo_cells

AnnData object with n_obs × n_vars = 4934 × 34643
    obs: 'DATASET', 'primer_type'
    var: 'gene_ids', 'feature_types'

In [43]:
adata_remaining_subset

AnnData object with n_obs × n_vars = 1320 × 34643
    obs: 'DATASET', 'primer_type'
    var: 'gene_ids', 'feature_types'

In [44]:
4760+6440

11200

In [45]:
adata_remaining_subset = adata_remaining_subset[adata_remaining_subset.obs_names.argsort()].copy()

In [46]:
# assert np.array_equal(adata_remaining_subset.obs_names, adata_matching_sorted.obs_names), "Row names are not the same"
assert np.array_equal(adata_remaining_subset.var_names, adata_matching_sorted.var_names), "Column names are not the same"

# Sum the .X matrices (count data) from both objects
summed_matrix = adata_remaining_subset.X + adata_matching_sorted.X

# Create a new AnnData object with the summed matrix
adata_dupl = ad.AnnData(X=summed_matrix, 
                        obs=adata_remaining_subset.obs,  # Use the same obs (row metadata)
                        var=adata_remaining_subset.var)  # Use the same var (column metadata)

# Print the shape to confirm the result
print(f"Shape of adata_dupl: {adata_dupl.shape}")

Shape of adata_dupl: (1320, 34643)


In [47]:
adata_dupl

AnnData object with n_obs × n_vars = 1320 × 34643
    obs: 'DATASET', 'primer_type'
    var: 'gene_ids', 'feature_types'

In [48]:
assert np.array_equal(adata_dupl.var_names, adata_solo_cells.var_names), "Column names are not the same"

final_adata = ad.concat([adata_dupl, adata_solo_cells])

Final_adata with the collapsed random hexamers

In [49]:
final_adata

AnnData object with n_obs × n_vars = 6254 × 34643
    obs: 'DATASET', 'primer_type'

In [50]:
final_adata.write(adata_out_mm)